In [50]:
import os.path as op

import hnn_core
from hnn_core import simulate_dipole, read_params, Network

hnn_core_root = op.join(op.dirname(hnn_core.__file__))

params_fname = op.join(hnn_core_root, 'param', 'default.json')
params = read_params(params_fname)

In [51]:
from ipywidgets import AppLayout, Button

In [52]:
# HNN Banner
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='10', width='auto'))

header_button = create_expanded_button('Human Neocortical Neurosolver', 'success')

In [105]:
# Sliders to change Params
from ipywidgets import (FloatSlider, Dropdown, Layout, Accordion, Tab, VBox,
                        interactive_output, interactive)
from functools import partial
  
def update_params(**updates):
    params.update(dict(**updates))
    return params
    
def _get_min(v):
    if v < 0:
        return v * 10
    else:
        return v * 0.1

def _get_max(v):
    if v > 0:
        return v * 10
    else:
        return v * 0.1

    
variables = dict(net=None, dpl=None)

def _get_sliders(params, param_keys):
    """Get sliders"""
    style = {'description_width': 'initial'}
    sliders = list()
    for d in param_keys:
        min_val = _get_min(params[d])
        max_val = _get_max(params[d])
        step = (max_val - min_val) / 10.
        slider = FloatSlider(
        value=params[d], min=min_val, max=max_val, step=step,
        description=d,
        disabled=False, continuous_update=False, orientation='horizontal',
        readout=True, readout_format='.2e',
        style=style)
        sliders.append(slider)
    
    interactive_output(update_params, {s.description: s for s in sliders})
    return sliders

sliders = [_get_sliders(params, ['gbar_L2Pyr_L2Pyr_ampa', 'gbar_L2Pyr_L2Pyr_nmda',
                                 'gbar_L2Basket_L2Pyr_gabaa', 'gbar_L2Basket_L2Pyr_gabab']),
           _get_sliders(params, ['gbar_L2Pyr_L5Pyr', 'gbar_L2Basket_L5Pyr', 'gbar_L5Pyr_L5Pyr_ampa',
                                 'gbar_L5Pyr_L5Pyr_nmda', 'gbar_L5Basket_L5Pyr_gabaa',
                                 'gbar_L5Basket_L5Pyr_gabab']),
           _get_sliders(params, ['gbar_L2Pyr_L2Basket', 'gbar_L2Basket_L2Basket']),
           _get_sliders(params, ['gbar_L2Pyr_L5Pyr', 'gbar_L2Basket_L5Pyr'])]

# accordians
boxes = [VBox(slider) for slider in sliders]
titles = ['Layer 2/3 Pyr', 'Layer 5 Pyr', 'Layer 2 Bas', 'Layer 5 Bas']
accordian = Accordion(children=boxes)
[accordian.set_title(idx, title) for idx, title in enumerate(titles)]

# Tabs
left_tab = Tab()
left_tab.children = [accordian, accordian]
titles = ['Cell connectivity', 'Drives']
[left_tab.set_title(idx, title) for idx, title in enumerate(titles)]  # XXX: why doesn't this work?

[None, None]

In [106]:
from ipywidgets import HBox, VBox

# Dropdown menu to switch between plots

def update_plot(plot_type):
    plot_out.clear_output()

    if not (plot_type['type'] == 'change' and plot_type['name'] == 'value'):
        return
    
    with plot_out:
        if plot_type['new'] == 'spikes':
            variables['net'].cell_response.plot_spikes_raster()
        elif plot_type['new'] == 'dipole current':
            variables['dpls'][0].plot()
        elif plot_type['new'] == 'input histogram':
            variables['net'].cell_response.plot_spikes_hist()


dropdown = Dropdown(
    options=['input histogram', 'dipole current', 'spikes'],
    value='dipole current',
    description='Plot:',
    disabled=False,
)

interactive(update_plot, plot_type='dipole current')
dropdown.observe(update_plot, 'value')

In [107]:
# Run button
from ipywidgets import Output

log_out = Output(layout={'border': '1px solid gray'})
plot_out = Output(layout={'border': '1px solid gray'})


def on_button_clicked(b, variables):
    """Run the simulation and plot outputs."""
    with log_out:
        variables['net'] = Network(params, add_drives_from_params=True)
        variables['dpls'] = simulate_dipole(variables['net'], n_trials=1)
    with plot_out:
        variables['dpls'][0].plot()

run_button = create_expanded_button('Run', 'success')
run_button.on_click(partial(on_button_clicked, variables=variables))

footer = HBox([run_button, dropdown])

In [108]:
# Final layout of the app
AppLayout(header=header_button,
          left_sidebar=left_tab,
          center=log_out,
          right_sidebar=plot_out,
          footer=footer,
          pane_widths=[1, 1, 1])

AppLayout(children=(Button(button_style='success', description='Human Neocortical Neurosolver', layout=Layout(…